# Coursera Capstone Project - Week 3

### Step 1: Imports

In [23]:
from IPython.display import Image as IPythonImage
from IPython.display import display_html
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import requests
import pandas as pd

### Step 2: Set the Wikipedia link and grab the HTML

In [24]:
#assign the Wikipedia link
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#grab the html from the Wikipedia page
raw_wikipedia_page = requests.get(wikipedia_link)
#assign the html as a variable
page = raw_wikipedia_page.text

### Step 3: read the html tables into a pandas dataframe

In [25]:
dfs=pd.read_html(page,header=0)

### Step 4: Take the first table from the Wikipedia page to use as our subject table

In [26]:
PC_Canada=dfs[0]

### Step 6: rename the columns

In [27]:
#rename the columns
PC_Canada.rename(columns={'Postcode' : 'PostalCode', 'Neighbourhood' : 'Neighborhood'},inplace=True)

### Step 7: Drop all rows where the Borough is not assigned

In [28]:
#Drop all rows where Borough is not assigned
NA = PC_Canada[PC_Canada['Borough'] == 'Not assigned'].index
PC_Canada.drop(NA,inplace=True)
#Reset the index
PC_Canada.reset_index(drop=True, inplace=True)

### Step 8: Where Neighborhood is not assigned, the neighborhood name = borough name

In [29]:
NNA=PC_Canada[PC_Canada['Neighborhood']=='Not assigned'].index
for n in NNA:
    PC_Canada['Neighborhood'].loc[[n]]=PC_Canada['Borough'].loc[[n]]

### Step 9: Where multiple rows exist for the same postal code, combine the rows and list all neighborhoods in the same cell

In [30]:
PC_Canada=PC_Canada.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

## Results

In [31]:
PC_Canada.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [32]:
print(PC_Canada.shape)

(103, 3)


# Part 2 - Add Latitude and Longitude

## Using the .csv file

### Step 1: Load the csv file and rename the postal code column

In [33]:
#Load the csv file
LLData = 'http://cocl.us/Geospatial_data'
LLDataDF=pd.read_csv(LLData, engine='python')

In [34]:
#rename the postal code column
LLDataDF.rename(columns={'Postal Code' : 'PostalCode'},inplace=True)

### Step 2: Merge the coordinate table with the main dataframe

In [35]:
#Merge the table of coordinates with the dataframe of boroughs and neighborhoods
PC_Canada=PC_Canada.merge(LLDataDF,on='PostalCode')

### Results

In [36]:
PC_Canada.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
